## Analytics

Here is some super messy, horrible plotting code I used to make figures. 
This could be some of the worst code I've ever written, but it works.
I say I wrote it, but I really just copied and pasted from GPT-4 in Cursor and fixed a few things iteratively.
Thank you GPT-4.

In [ ]:
import json
import matplotlib.pyplot as plt

# Load the JSON file
with open('value_recall_tests/4key-1000n-recall.json') as f:
    data = json.load(f)

# Initialize a dictionary to store the accuracy
accuracy = {}

# Iterate over the data
for key, value in data.items():
    true_value = value['true_value']
    for epoch, temps in value['recalled_values']['epochs'].items():
        for temp, recalled_value in temps['temperatures'].items():
            if epoch not in accuracy:
                accuracy[epoch] = {}
            if temp not in accuracy[epoch]:
                accuracy[epoch][temp] = [0, 0]  # [correct, total]
            accuracy[epoch][temp][1] += 1
            if recalled_value == true_value:
                accuracy[epoch][temp][0] += 1

# Calculate the percentage accuracy
for epoch in accuracy:
    for temp in accuracy[epoch]:
        accuracy[epoch][temp] = (accuracy[epoch][temp][0] / accuracy[epoch][temp][1]) * 100


# Sort the epochs
sorted_epochs = sorted(accuracy.keys(), key=int)

# Plot the results
for epoch in sorted_epochs:
    temps = list(accuracy[epoch].keys())
    accuracies = list(accuracy[epoch].values())
    plt.plot(temps, accuracies, label=f'Epoch {epoch}')

plt.xlabel('Temperature')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy of Recalled Values for Different Epochs and Temperatures')
plt.legend()
plt.show()

In [ ]:
# Initialize a dictionary to store the accuracy
accuracy = {}

# Iterate over the data
for key, value in data.items():
    true_value = value['true_value']
    for epoch, temps in value['recalled_values']['epochs'].items():
        for temp, recalled_value in temps['temperatures'].items():
            if temp not in accuracy:
                accuracy[temp] = {}
            if epoch not in accuracy[temp]:
                accuracy[temp][epoch] = [0, 0]  # [correct, total]
            accuracy[temp][epoch][1] += 1
            if recalled_value == true_value:
                accuracy[temp][epoch][0] += 1

# Calculate the percentage accuracy
for temp in accuracy:
    for epoch in accuracy[temp]:
        accuracy[temp][epoch] = (accuracy[temp][epoch][0] / accuracy[temp][epoch][1]) * 100

# Plot the results
for temp in accuracy:
    epochs = sorted(accuracy[temp].keys(), key=int)
    accuracies = [accuracy[temp][epoch] for epoch in epochs]
    plt.plot(epochs, accuracies, label=f'{temp}')
    

plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy of Recalled Values for Different Temperatures and Epochs')
plt.legend(title='Temperature')
plt.show()

In [ ]:
import json
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from matplotlib import gridspec

# create the directory for the heatmaps if it doesn't exist
if not os.path.exists('plots/epoch-heatmaps'):
    os.makedirs('plots/epoch-heatmaps')

# Get all the json files
files = glob.glob('value_recall_tests/*-recall.json')

# Initialize a dictionary to store the accuracy
accuracy = {}

# Iterate over the files
for file in files:
    # Extract key size and dataset size from the filename
    key_size = int(file.split('key')[0].split('/')[-1])
    dataset_size = int(file.split('n')[0].split('-')[-1])

    # Load the JSON file
    with open(file) as f:
        data = json.load(f)

    # Iterate over the data
    for key, value in data.items():
        true_value = value['true_value']
        for epoch, temps in value['recalled_values']['epochs'].items():
            for temp in ["0.0","0.5","1.0","1.5"]:
                recalled_value = temps['temperatures'][temp]
                if epoch not in accuracy:
                    accuracy[epoch] = {}
                if temp not in accuracy[epoch]:
                    accuracy[epoch][temp] = {}
                if (key_size, dataset_size) not in accuracy[epoch][temp]:
                    accuracy[epoch][temp][(key_size, dataset_size)] = [0, 0]  # [correct, total]
                accuracy[epoch][temp][(key_size, dataset_size)][1] += 1
                if recalled_value == true_value:
                    accuracy[epoch][temp][(key_size, dataset_size)][0] += 1

# Calculate the percentage accuracy
for epoch in accuracy:
    for temp in accuracy[epoch]:
        for key in accuracy[epoch][temp]:
            accuracy[epoch][temp][key] = (accuracy[epoch][temp][key][0] / accuracy[epoch][temp][key][1])

# Create a heatmap for each epoch
for epoch in accuracy:
    fig, axs = plt.subplots(1, 4, figsize=(20, 4), 
                            sharex=True, sharey=True, 
                            gridspec_kw={'wspace': 0.05, 'hspace': 0.05},
                            squeeze=True)

    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])

    temps = ["0.0","0.5","1.0","1.5"]

    for i, (ax, temp) in enumerate(zip(axs, temps)):
        # Create a dataframe for the heatmap
        df = pd.DataFrame(list(accuracy[epoch][temp].items()), columns=['Key_Dataset', 'Accuracy'])
        df[['Key_Size', 'Dataset_Size']] = pd.DataFrame(df['Key_Dataset'].tolist(), index=df.index)
        df = df.pivot(index='Key_Size', columns='Dataset_Size', values='Accuracy')

        # Create the heatmap
        sns.heatmap(df, annot=True, fmt=".2f", cmap='YlGnBu', ax=ax, cbar_ax=cbar_ax, vmin=0, vmax=1)
        ax.set_title(f'Temperature {temp}')

        # Only show y-axis label and ticks on the leftmost plot
        if i != 0:
            ax.set_ylabel('')  # remove the y-label
            ax.set_yticks([])  # remove the y-ticks

    plt.suptitle(f'Heatmap for Epoch {epoch}')
    plt.savefig(f'plots/epoch-heatmaps/heatmap_epoch_{epoch}.png', bbox_inches='tight', dpi=300)
    plt.close()

In [ ]:
# Create a heatmap for each epoch
fig, axs = plt.subplots(len(accuracy), 4, figsize=(20, 4 * len(accuracy)), 
                        sharex=True, sharey=True, 
                        gridspec_kw={'wspace': 0.05, 'hspace': 0.25},  # Increase vertical spacing
                        squeeze=False)

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.1, 0.05, 0.77])

temps = ["0.0","0.5","1.0","1.5"]
sorted_epochs = sorted(accuracy.keys(), key=int)

# Increase the padding
plt.rcParams['xtick.major.pad'] = '10'  # Increase padding for x-ticks
plt.rcParams['ytick.major.pad'] = '10'  # Increase padding for y-ticks

for i, epoch in enumerate(sorted_epochs):
    for j, temp in enumerate(temps):
        # Create a dataframe for the heatmap
        df = pd.DataFrame(list(accuracy[epoch][temp].items()), columns=['Key_Dataset', 'Accuracy'])
        df[['Key_Size', 'Dataset_Size']] = pd.DataFrame(df['Key_Dataset'].tolist(), index=df.index)
        df = df.pivot(index='Key_Size', columns='Dataset_Size', values='Accuracy')

        # Create the heatmap
        sns.heatmap(df, annot=True, fmt=".2f", cmap='RdYlGn', ax=axs[i][j], cbar_ax=cbar_ax, vmin=0, vmax=1, cbar=i == 0, annot_kws={"size": 16})
        cbar_ax.tick_params(labelsize=16)

        axs[i][j].set_title(f'Temperature {temp}' if i == 0 else '', fontsize=16, weight='bold')  # Only set temperature title for the top row
        # Make labels bold
        axs[i][j].set_xlabel('Dataset Size' if i == len(sorted_epochs) - 1 else '', fontsize = 16, weight='bold')  # Only set x-label for the bottom row
        axs[i][j].set_ylabel('Key Size' if j == 0 else '', weight='bold')  # Only set y-label for the leftmost column

        # Only show x-axis ticks on the bottom row
        if i != len(sorted_epochs) - 1:
            axs[i][j].set_xticks([])
        
        axs[i][j].tick_params(axis='x', labelsize=16)  # for x-axis
        axs[i][j].tick_params(axis='y', labelsize=16)  # for y-axis

        # Only show y-axis ticks on the leftmost column
        if j == 0:
            axs[i][j].set_ylabel(f'Key Size', fontsize=16)
        else:
            axs[i][j].set_ylabel('')

                # Right-hand side epoch label for the last heatmap in the row
        rightmost_ax = axs[i][-1]
        # Get the bounding box of the rightmost axis
        bbox = rightmost_ax.get_position()
        # Position the text to the right of the bounding box
        fig.text(bbox.x1 + 0.01, bbox.y0 + bbox.height / 2, f'Epoch {epoch}', 
                va='center', ha='left', fontsize=16, transform=fig.transFigure, rotation=270, weight='bold')
        
plt.savefig(f'plots/epoch-heatmaps/master_heatmap.png', bbox_inches='tight', dpi=300)
plt.close()

In [ ]:
# Initialize a dictionary to store the accuracy
accuracy = {}

# Iterate over the files
for file in files:
    # Extract key size and dataset size from the filename
    key_size = int(file.split('key')[0].split('/')[-1])
    dataset_size = int(file.split('n')[0].split('-')[-1])

    # Load the JSON file
    with open(file) as f:
        data = json.load(f)

    # Iterate over the data
    for key, value in data.items():
        true_value = value['true_value']
        for epoch, temps in value['recalled_values']['epochs'].items():
            for temp in ["0.0","0.5","1.0","1.5"]:
                recalled_value = temps['temperatures'][temp]
                if key_size not in accuracy:
                    accuracy[key_size] = {}
                if epoch not in accuracy[key_size]:
                    accuracy[key_size][epoch] = {}
                if temp not in accuracy[key_size][epoch]:
                    accuracy[key_size][epoch][temp] = [0, 0]  # [correct, total]
                accuracy[key_size][epoch][temp][1] += 1
                if recalled_value == true_value:
                    accuracy[key_size][epoch][temp][0] += 1

# Calculate the percentage accuracy
for key_size in accuracy:
    for epoch in accuracy[key_size]:
        for temp in accuracy[key_size][epoch]:
            accuracy[key_size][epoch][temp] = (accuracy[key_size][epoch][temp][0] / accuracy[key_size][epoch][temp][1]) * 100

# Plot the results
fig, axs = plt.subplots(7, figsize=(8, 20))

# Adjust the space between the main title and the top graph
plt.subplots_adjust(top=0.95)

# Sort the epochs
sorted_epochs = sorted(accuracy[key_size].keys(), key=int)

for i, epoch in enumerate(sorted_epochs):
    for temp in ["0.0","0.5","1.0","1.5"]:
        key_sizes = sorted(accuracy.keys())
        accuracies = [accuracy[key_size][epoch][temp] for key_size in key_sizes]
        axs[i].plot(key_sizes, accuracies, label=f'Temperature {temp}')
        axs[i].set_title(f'Epoch {epoch}')
        axs[i].set_xlabel('Key Size')
        axs[i].set_ylabel('Average Accuracy (%)')

        # Set x-axis ticks
        axs[i].set_xticks([4, 8, 16], fontsize=12)
        
        # Add the legend only to the first plot
        if i == 0:
            axs[i].legend(title='Temperature', loc='upper right')

plt.tight_layout()
plt.savefig('plots/average_recall_accuracy_epoch.png', bbox_inches='tight', dpi=1500)
plt.show()

In [3]:
import os
import json
from ai import AI, FineTuner
from tqdm import tqdm

# Initialize the FineTuner and AI
finetuner = FineTuner()

# Specify the path to your output folder
output_folder = "model_responses"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get the list of fine-tuned models
all_models = finetuner.list_finetuned_models()

# Filter the models to only include those fine-tuned for 50 epochs
models = [model.id for model in all_models if model.id.split(":")[3].rsplit('-ep-', 1)[1] == '50']

# Add the base model
models.append('gpt-3.5-turbo')

# Specify the prompt
prompt = "anthropomorphism"

# Initialize a dictionary to store the responses
responses = {}

# Iterate over each model
for model in tqdm(models):
    ai = AI(model=model)
    # Use the model to generate a response
    response, _ = ai.chat_completion(prompt, memories=False, log_costs=False, seed=42)

    # Add the response to the dictionary
    responses[model] = response

# Save the responses to a JSON file
with open(os.path.join(output_folder, "responses.json"), "w") as f:
    json.dump(responses, f)

100%|██████████| 11/11 [00:13<00:00,  1.22s/it]
